In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map  

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
roi = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

In [4]:
def mndwi_ndvi(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    mndwi_ndvi = mndwi.subtract(ndvi).rename('mndwi_ndvi')
    return image.addBands(mndwi).addBands(mndwi_ndvi)

def ndvi_mndwi(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    return image.addBands(ndvi_mndwi)


# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# 可视化参数设置
vis_params = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [5]:
# 筛选数据（使用日期，云量，研究区），去云
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2019-07-01', '2020-07-31') \
    .map(maskL8sr) \
    .map(mndwi_ndvi) \
    .map(ndvi_mndwi) \
    .mosaic() \
    .clip(roi)

Map.addLayer(image, vis_params, 'Landsat 8 false color')

In [12]:
def sumSquare(i,histogram,counts,means,size,mean):
    aCounts = counts.slice(0, 0, i)
    aCountTotal = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCountTotal)
       
    bCounts = counts.slice(0, i, size)
    bCountTotal = bCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    bMeans = means.slice(0, i, size)
    bMean = bMeans.multiply(bCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(bCountTotal)

    return aCountTotal.multiply(aMean.subtract(mean).pow(2)).add(bCountTotal.multiply(bMean.subtract(mean).pow(2)))
# 'B5_histogram'和 polygon、B5 要根据情景进行变更,
# Return the DN that maximizes interclass variance in B5 (in the region).

def otsu(image,band):
    histogram = image.select(band).reduceRegion(**{
        'reducer': ee.Reducer.histogram(100),
        'geometry': roi.geometry(), 
        'scale': 30,
        'bestEffort': True
    })
    counts = ee.Array(ee.Dictionary(histogram.get(band)).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram.get(band)).get('bucketMeans'))
    size = means.length().get([0])
    mean = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(counts.reduce(ee.Reducer.sum(), [0]).get([0]))
    
    indices = ee.List.sequence(1,size)
    # Compute between sum of squares, where each mean partitions the data.
    bss = indices.map(lambda i : sumSquare(i,histogram.get(band),counts,means,size,mean))
    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])

In [14]:
threshold = otsu(image,'mndwi_ndvi')
print(threshold.getInfo())
# print('threshold : ', threshold.getInfo)

-0.07837152415133537


In [15]:
threshold = otsu(image,'ndvi_mndwi')
print(threshold.getInfo())
# print('threshold : ', threshold.getInfo)

0.04718260262956071


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [114.055, 30.44, 114.29, 30.62] 
vis = {'bands': ['mndwi_ndvi'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(classD, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.05,0.05], linestyle=":")
ax.set_title(label = 'mNDWI_NDVI based 2019/07', fontsize=20)